In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [5]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [6]:
# 25 EV sessions per weekday
events_25_ev = get_synth_events(25)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Checking session details
session_copy = deepcopy(events_25_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

646 session_10 646 763 10.518997336053149 Plugin
649 session_2 649 772 69.31884196506515 Plugin
648 session_11 648 754 12.627502905540887 Plugin
651 session_17 651 765 6.708658900683705 Plugin
664 session_19 664 731 22.065660079498876 Plugin
650 session_5 650 746 9.584748308529516 Plugin
727 session_6 727 791 1.8436599105548768 Plugin
663 session_15 663 772 8.3116119745761 Plugin
658 session_16 658 786 5.369541961619029 Plugin
665 session_7 665 723 5.272934799191699 Plugin
666 session_22 666 773 14.561275160988497 Plugin
663 session_1 663 777 27.292856586470073 Plugin
661 session_12 661 769 19.362337683607223 Plugin
763 session_13 763 818 10.430032707381972 Plugin
759 session_14 759 813 8.539172304682744 Plugin
700 session_0 700 724 5.910845734627817 Plugin
667 session_3 667 732 18.14408941521955 Plugin
737 session_8 737 767 15.055163978948357 Plugin
671 session_18 671 778 2.8493975249027863 Plugin
669 session_9 669 796 7.0597084167695945 Plugin
731 session_20 731 749 9.375298382932423

#### Algorithms and new equation

In [8]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

#### Run Simulation with New Equation

In [11]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2024, 10, 1))
    end = timezone.localize(datetime(2024, 10, 1))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_25_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_4", "session_12", "session_2"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [12]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.3341703414916992
('RR', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 1.583069086074829
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 12.830713748931885
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.3167901039123535
('RR', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.6161415576934814
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 12.72611141204834
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.2804441452026367
('RR', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 1.6424553394317627
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 12.566665410995483
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.3006105422973633
('RR', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 1.7000865936279297
('MPC', 75)
60.240

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [13]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Energy Delivered to priority evs (%)": analysis.proportion_of_priority_evs_energy_delivered(sim, [ "session_3",
                                                                                                            "session_5",
                                                                                                            "session_10",
                                                                                                            "session_12",
                                                                                                            "session_21",]),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [14]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [15]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,165.000,24.161,336.344,100.000,99.000,0.382
1,three_phase,RR,60,85.101,76.0,87.500,21.211,286.232,71.374,52.500,0.417
2,three_phase,MPC,60,71.498,32.0,98.557,17.792,240.478,53.131,59.134,0.419
3,three_phase,Unctrl,65,100.000,100.0,152.308,24.161,336.344,100.000,99.000,0.395
4,three_phase,RR,65,80.954,68.0,85.877,19.934,272.284,55.633,55.820,0.420
5,three_phase,MPC,65,75.477,28.0,95.777,18.943,253.864,71.746,62.255,0.418
6,three_phase,Unctrl,70,100.000,100.0,141.429,24.161,336.344,100.000,99.000,0.364
7,three_phase,RR,70,82.944,76.0,78.571,20.492,278.976,61.703,55.000,0.393
8,three_phase,MPC,70,75.421,24.0,96.050,18.772,253.674,71.723,67.235,0.446
9,three_phase,Unctrl,75,100.000,100.0,132.000,24.161,336.344,100.000,99.000,0.356


#### Simulation with different algorithms

In [27]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [28]:
sims = dict()

In [29]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.28501200675964355
('LLF', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.8095314502716064
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 12.403924226760864
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.27858734130859375
('LLF', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.774163007736206
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 12.962316751480103
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.31701135635375977
('LLF', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.7493212223052979
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 12.566829681396484
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.3005049228668213
('LLF', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.7002122402191162
('MPC', 75

In [30]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [31]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,165.000,24.161,336.344,100.000,99.000,0.376
1,three_phase,LLF,60,82.949,76.0,79.208,20.677,278.992,61.703,47.525,0.398
2,three_phase,MPC,60,75.313,24.0,92.692,18.897,253.312,71.234,55.615,0.406
3,three_phase,Unctrl,65,100.000,100.0,152.308,24.161,336.344,100.000,99.000,0.380
4,three_phase,LLF,65,82.917,72.0,83.015,20.550,278.887,61.586,53.960,0.377
5,three_phase,MPC,65,75.422,24.0,99.283,18.910,253.678,71.728,64.534,0.398
6,three_phase,Unctrl,70,100.000,100.0,141.429,24.161,336.344,100.000,99.000,0.381
7,three_phase,LLF,70,82.932,76.0,74.714,20.518,278.938,61.703,52.300,0.393
8,three_phase,MPC,70,75.411,28.0,99.014,18.817,253.639,71.726,69.310,0.429
9,three_phase,Unctrl,75,100.000,100.0,132.000,24.161,336.344,100.000,99.000,0.354
